In [283]:
import pandas as pd
import os
import plotly.graph_objs as go
import plotly.express as px
from copy import deepcopy
from plotly.subplots import make_subplots


In [24]:
df1 = pd.read_excel('Coronavirus_DailyTrendsdata.xlsx',  sheet_name='US', header=1)

In [25]:
delnames = [i for i in df1.columns if 'City' in i]
df1.drop(delnames, inplace=True, axis=1)

In [29]:
df1 = df1.dropna(how='all')

### bar chart race

In [126]:
basepath = 'data/'
files = os.listdir(basepath)

In [127]:
total = pd.DataFrame()
temp = pd.read_csv(basepath +files[0], skiprows = 1)
total['Day'] = temp['Day']    

In [128]:
for i in files:
    a = pd.read_csv(basepath + i, skiprows = 1)
    colname = a.columns[1]
    a.loc[a[colname] == '<1',colname] = 0
    newName = colname.split(":")[0]
    total[newName] = a[colname]

In [129]:
#total.cumsum()
total = total.fillna(0)
total.iloc[:,1:] = total.iloc[:,1:].apply(pd.to_numeric)
#total.iloc[:,1:] = total.iloc[:,1:].cumsum()
#total.transpose().to_csv("NoRunningSum.csv")

In [130]:
#total.to_csv("cleanedData.csv")

In [131]:
#total.transpose().to_csv("cleanedDataAlternate.csv")

### line

In [169]:
basepath2 = 'generalTerms/'
files2 = os.listdir(basepath2)

In [365]:
plotnames = []
for i in files2:
    a = pd.read_csv(basepath2 + i, skiprows = 1)
    colname = a.columns[1]
    plotnames.append(colname.split(':')[0])

In [417]:
#for i in files2:
def getData(i):
    a = pd.read_csv(basepath2 + i, skiprows = 1)
    a[['Year','Month', 'Day']] = a.Week.str.split('-', expand=True)
    colname = a.columns[1]
    a.columns.values[1] = colname.split(':')[0]
    #colname2 = a.columns[0].split('-')[1:]
    #a.columns.values[0] = "-".join(colname2)
    a = a[(a['Year'] == '2019') | (a['Year'] == '2020')]
    a = a[(a['Month'] == '01') | (a['Month'] == '02') | (a['Month'] == '03') | (a['Month'] == '04')]
    #a['Week'] = pd.to_datetime(a.Week)
    a['Week'] = ["/".join(x.split('-')[1:]) for x in a['Week']]
    df1 = a[(a['Year'] == '2019')]
    df2 = a[(a['Year'] == '2020')]
    df1.loc[:,'Week'] = df2.Week.values
    return df1 , df2

In [429]:
fig = make_subplots(rows=3, cols=4, subplot_titles=plotnames)

df2019, df2020= getData(files2[0])
traceapp1 = go.Scatter(x = df2019.Week, y = df2019.iloc[:,1],
                name = '2019',
                mode='lines',
                legendgroup='group1',
                line = dict(width = 1,
                            color = 'rgb(111, 231, 219)'),
                fill='tozeroy')

traceapp2 = go.Scatter(x = df2020.Week, y = df2020.iloc[:,1],
                    name = '2020',
                    legendgroup='group1',
                    mode='lines',
                    line = dict(width = 1,
                                color = 'red'),
                    fill='tozeroy')

fig.add_trace(traceapp2,
              row=1, col=1)
fig.add_trace(traceapp1,
              row=1, col=1)
fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=10), row=1, col=1)


for i in range(1, len(files2)):
    df2019, df2020= getData(files2[i])
    traceapp1 = go.Scatter(x = df2019.Week, y = df2019.iloc[:,1],
                    name = '2019',
                    mode='lines',
                    line = dict(width = 1,
                                color = 'rgb(111, 231, 219)'),
                    fill='tozeroy',
                    showlegend= False)

    traceapp2 = go.Scatter(x = df2020.Week, y = df2020.iloc[:,1],
                        name = '2020',
                        mode='lines',
                        line = dict(width = 1,
                                    color = 'red'),
                        fill='tozeroy',
                        showlegend= False)

    if i <= 3:
        fig.add_trace(traceapp2,
                      row=1, col=i+1)
        fig.add_trace(traceapp1,
                      row=1, col=i+1)
        fig.update_xaxes(tickangle=45, tickfont=dict(family='Rockwell', color='black', size=10), row=1, col=i+1)
    elif 4 <= i <= 7:
        fig.add_trace(traceapp2,
                      row=2, col=i-3)
        fig.add_trace(traceapp1,
                      row=2, col=i-3)
        fig.update_xaxes(tickangle=45,tickfont=dict(family='Rockwell', color='black', size=10), row=2, col=i-3)
    elif 8 <= i <= 11:
        fig.add_trace(traceapp2,
                      row=3, col=i-7)
        fig.add_trace(traceapp1,
                      row=3, col=i-7)
        fig.update_xaxes(tickangle=45,tickfont=dict(family='Rockwell', color='black', size=10), row=3, col=i-7)
        

In [430]:
# all_trace = [traceapp1, traceapp2] 
# all_figure = go.Figure(data = all_trace, layout = all_layout)
fig.update_layout(hovermode = 'x', showlegend= True, title = "Google Search Interest Overtime")

fig.show()